In [ ]:
# Comprobamos si está tensorflow-gpu==2.3.0
# !pip freeze

In [ ]:
# Hay que instalar esto si se quiere utilizar la gpu
#!pip install tensorflow-gpu==2.3.0 

In [ ]:
# Esta es la tarjeta grafica
# !nvidia-smi

In [1]:
# instalar librerías. Esta casilla es últil por ejemplo si se ejecuta el cuaderno en Google Colab
# Note que existen otras dependencias como tensorflow, etc. que en este caso se encontrarían ya instaladas
%%capture
!pip install transformers==4.2.1
!pip install tensorflow-gpu==2.3.0 
print('Done!')

In [3]:
%reset
import pandas as pd
import tensorflow as tf
from transformers import DistilBertConfig, TFDistilBertForSequenceClassification, DistilBertTokenizer
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Para trabajar teniendo encuenta el desbalanceo
from sklearn.utils import class_weight
import os
import tqdm
import numpy as np
from tensorflow.keras import backend as K

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [4]:
train_dataframe = pd.read_csv("https://raw.githubusercontent.com/jibt1/competition_group/main/datasets/haha_2021_train.csv", sep=',')
train_dataframe.head()

,id,text,is_humor,votes_no,votes_1,votes_2,votes_3,votes_4,votes_5,humor_rating,humor_mechanism,humor_target
0,tweet1,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,1,1,2,2,0,0,0,1.5,NaN,NaN
1,tweet2,"—Vamos Luke desenfunda tu sable, demuestra tu ...",1,1,3,0,1,0,0,1.5,NaN,NaN
2,tweet3,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...",1,0,2,1,0,1,1,2.6,NaN,NaN
3,tweet4,No se porqué me hago la cabeza deooos,0,3,0,0,0,0,0,NaN,NaN,NaN
4,tweet5,Quisiera saber que hago durante la siesta de l...,0,4,0,1,0,0,0,NaN,NaN,NaN


In [6]:
test_dataframe = pd.read_csv("https://raw.githubusercontent.com/jibt1/competition_group/main/datasets/haha_2021_dev.csv", sep=',')
test_dataframe.head() # Jaja qué cabrones, está sin etiquetar. Toca sacar la validación del conjunto de entrenamiento

,id,text
0,tweet24001,Historia #36.\n*Entra corriendo y gritando al ...
1,tweet24002,estoy tomando helado en el centro y pedi tiram...
2,tweet24003,— ¿De 15 o 20 centímetros?\n\n— Ven y averígua...
3,tweet24004,Que fea soy
4,tweet24005,"#OMG, VAMOS EN RETROCESO RESULTE CON 40 SEGUID..."


In [7]:
cfg = {}
cfg["framework"] = "tf"
cfg["max_length"] = 256 # 380 caracteros maximo por tweet (Mirar maximo de longitud)
cfg["transformer_model_name"] = "dccuchile/bert-base-spanish-wwm-cased" # Este es el modelo Bert para Spanish, con mayusculas
cfg["num_labels"] = 2

In [100]:
# dim dimension del pooling layer de los outputs del encoder en la salida de la ultima capa
# https://huggingface.co/transformers/model_doc/distilbert.html
# https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased
# dropout es el dropout de las denses de las capas de encoders
# seq_classif_dropout es el dropout de la ultima densa ajena a Bert
# Reducimos la complejidad del problema, solo tenemos 25000 tweets

config_bert = DistilBertConfig(num_labels = cfg["num_labels"], 
                               attention_dropout=0.75, seq_classif_dropout=0, dropout=0.75,
                               n_heads=4, dim=64, max_position_embeddings=cfg["max_length"],
                               n_layers=2, hidden_dim=128)
model = TFDistilBertForSequenceClassification.from_pretrained(cfg["transformer_model_name"], config=config_bert)

Some layers from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing TFDistilBertForSequenceClassification: ['mlm___cls', 'bert']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['pre_classifier', 'classifier', 'distilbert', 'dropout_103']
You should probably TRAIN this model on a down-stream task to be able to use it for predi

In [101]:
# Tripadvisor, aplicamos regresion sin reducir de 5 puntos a 0 1 2
# Cambiar las metricas, MSE

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1Score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [102]:
from tensorflow.keras.layers import Input, Softmax, Dense
from tensorflow.keras.models import Model

In [104]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True)
model.compile(optimizer=optimizer, loss=loss, metrics = [f1Score, 'accuracy'])
model.summary()

Model: "tf_distil_bert_for_sequence_classification_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  2036864   
_________________________________________________________________
pre_classifier (Dense)       multiple                  4160      
_________________________________________________________________
classifier (Dense)           multiple                  130       
_________________________________________________________________
dropout_103 (Dropout)        multiple                  0         
Total params: 2,041,154
Trainable params: 2,041,154
Non-trainable params: 0
_________________________________________________________________


In [105]:
# Cargamos el tokenizador correspondiente ¿Lematiza?
cfg['tokenizer'] = DistilBertTokenizer.from_pretrained(cfg['transformer_model_name'] )
# Proceso de scikit learn para hacer OHE a 0 1 de la salida
cfg['label_binarizer'] = preprocessing.LabelBinarizer()

In [106]:
train, val = train_test_split(train_dataframe, test_size=.2)

In [14]:
# Empleamos el labelizador
#cfg['label_binarizer'].fit(train["is_humor"], num_labels=2)
train_blabels = to_categorical(train["is_humor"])
print(train_blabels)
train_blabels_t = tf.convert_to_tensor(train_blabels, dtype='int32')

val_blabels = to_categorical(val["is_humor"])
print(train_blabels)
val_blabels_t = tf.convert_to_tensor(val_blabels, dtype='int32')

# Clases ligeramente desbalanceadas
df_clases = pd.DataFrame(train_blabels)
round(df_clases.value_counts()/df_clases.shape[0] * 100, 2)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]


0    1  
1.0  0.0    61.43
0.0  1.0    38.57
dtype: float64

In [15]:
def get_model_inputs(cfg, data):
    encodings = cfg['tokenizer'](data, truncation=True, padding='max_length', max_length=cfg['max_length'], return_tensors=cfg['framework'])
    inputs = {'input_ids': encodings['input_ids'],
            'attention_mask': encodings['attention_mask']
            }
    return inputs

In [16]:
# Constriumos la matriz de diseño con sus correspondientes Masks
train_inputs = get_model_inputs(cfg, train["text"].to_list())
train_inputs

{'attention_mask': <tf.Tensor: shape=(19200, 256), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'input_ids': <tf.Tensor: shape=(19200, 256), dtype=int32, numpy=
 array([[    4,     3,  1067, ...,     1,     1,     1],
        [    4,  1149,  1139, ...,     1,     1,     1],
        [    4,  1339,  1240, ...,     1,     1,     1],
        ...,
        [    4,  1149, 11588, ...,     1,     1,     1],
        [    4, 19506,  1030, ...,     1,     1,     1],
        [    4,     3,  1139, ...,     1,     1,     1]], dtype=int32)>}

In [19]:
val_inputs = get_model_inputs(cfg, val["text"].to_list())
val_inputs

{'attention_mask': <tf.Tensor: shape=(4800, 256), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'input_ids': <tf.Tensor: shape=(4800, 256), dtype=int32, numpy=
 array([[    4,  1038,  3383, ...,     1,     1,     1],
        [    4,  2458, 26247, ...,     1,     1,     1],
        [    4,  1856,  1796, ...,     1,     1,     1],
        ...,
        [    4,  1369,  9554, ...,     1,     1,     1],
        [    4,  2358,  1033, ...,     1,     1,     1],
        [    4,  1261,  1987, ...,     1,     1,     1]], dtype=int32)>}

In [17]:
print(cfg['tokenizer'].pad_token)
cfg['tokenizer'].encode([cfg['tokenizer'].pad_token])

[PAD]


[4, 1, 5]

In [18]:
cfg['tokenizer'].decode([4, 1, 5])

'[CLS] [PAD] [SEP]'

In [20]:
cfg['checkpoints_dir'] = 'checkpoints'
cfg['model_name'] = 'distilbert-humor'
cfg['trained_model_name'] = os.path.join(cfg['checkpoints_dir'], cfg['model_name'])

In [21]:
tlabels = train_blabels.ravel()

In [107]:
epochs_max = 10
epochs_to_save = 5
batch_size = 32

class_weights = class_weight.compute_class_weight('balanced', np.unique(tlabels), tlabels)
class_weights = dict(enumerate(class_weights))

for epoch in tqdm.tqdm(range(0, epochs_max, epochs_to_save)):
    print(f'Training model, epochs {epoch+1} - {epoch+epochs_to_save}')
    
    # entrenar el modelo. Opcionalmente, se puede suministrar datos de validación => validation_data=(val_inputs,val_blabels_t )
    model.fit(train_inputs, y=train_blabels_t, epochs=epochs_to_save, 
              batch_size=batch_size, validation_data=(val_inputs, val_blabels_t), class_weight=class_weights)

    model.save_pretrained(cfg['trained_model_name'] + f'-epochs-{epoch+1:03d}-{epoch+epochs_to_save:03d}')
    cfg['tokenizer'].save_pretrained(cfg['trained_model_name'] + f'-epochs-{epoch+1:03d}-{epoch+epochs_to_save:03d}')










  0%|          | 0/2 [00:00<?, ?it/s]The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Training model, epochs 1 - 5
Epoch 1/5


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


599/600 [============================>.] - ETA: 0s - loss: 0.6307 - f1Score: 0.1704 - accuracy: 0.6459

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


600/600 [==============================] - 14s 24ms/step - loss: 0.6304 - f1Score: 0.1710 - accuracy: 0.6461 - val_loss: 0.5286 - val_f1Score: 0.5597 - val_accuracy: 0.7390
Epoch 2/5
600/600 [==============================] - 14s 23ms/step - loss: 0.4907 - f1Score: 0.6238 - accuracy: 0.7663 - val_loss: 0.4511 - val_f1Score: 0.7761 - val_accuracy: 0.8029
Epoch 3/5
600/600 [==============================] - 14s 23ms/step - loss: 0.4481 - f1Score: 0.7258 - accuracy: 0.7965 - val_loss: 0.4312 - val_f1Score: 0.7901 - val_accuracy: 0.8138
Epoch 4/5
600/600 [==============================] - 14s 23ms/step - loss: 0.4193 - f1Score: 0.7520 - accuracy: 0.8131 - val_loss: 0.4201 - val_f1Score: 0.7934 - val_accuracy: 0.8198
Epoch 5/5
600/600 [==============================] - 14s 23ms/step - loss: 0.4033 - f1Score: 0.7662 - accuracy: 0.8209 - val_loss: 0.4075 - val_f1Score: 0.8013 - val_accuracy: 0.8238











 50%|█████     | 1/2 [01:12<01:12, 72.57s/it]

Training model, epochs 6 - 10
Epoch 1/5
600/600 [==============================] - 14s 23ms/step - loss: 0.3866 - f1Score: 0.7802 - accuracy: 0.8348 - val_loss: 0.4102 - val_f1Score: 0.8066 - val_accuracy: 0.8296
Epoch 2/5
600/600 [==============================] - 14s 23ms/step - loss: 0.3678 - f1Score: 0.7954 - accuracy: 0.8417 - val_loss: 0.4124 - val_f1Score: 0.8115 - val_accuracy: 0.8308
Epoch 3/5
600/600 [==============================] - 14s 23ms/step - loss: 0.3569 - f1Score: 0.8027 - accuracy: 0.8470 - val_loss: 0.4087 - val_f1Score: 0.8138 - val_accuracy: 0.8323
Epoch 4/5
600/600 [==============================] - 14s 23ms/step - loss: 0.3437 - f1Score: 0.8138 - accuracy: 0.8559 - val_loss: 0.4051 - val_f1Score: 0.8127 - val_accuracy: 0.8313
Epoch 5/5
600/600 [==============================] - 14s 23ms/step - loss: 0.3353 - f1Score: 0.8176 - accuracy: 0.8608 - val_loss: 0.4146 - val_f1Score: 0.8149 - val_accuracy: 0.8313











100%|██████████| 2/2 [02:23<00:00, 71.53s/it]


In [108]:
def evaluate_model(y_true, y_pred, y_score=None, pos_label='positive'):
  print('==== Sumario de la clasificación ==== ')
  print(classification_report(y_true, y_pred))

  print('Accuracy -> {:.2%}\n'.format(accuracy_score(y_true, y_pred)))

  # graficar matriz de confusión
  display_labels = sorted(unique_labels(y_true, y_pred), reverse=True)
  cm = confusion_matrix(y_true, y_pred, labels=display_labels)

  z = cm[::-1]
  x = display_labels
  y =  x[::-1].copy()
  z_text = [[str(y) for y in x] for x in z]

  fig_cm = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

  fig_cm.update_layout(
      height=400, width=400,
      showlegend=True,
      margin={'t':150, 'l':0},
      title={'text' : 'Matriz de Confusión', 'x':0.5, 'xanchor': 'center'},
      xaxis = {'title_text':'Valor Real', 'tickangle':45, 'side':'top'},
      yaxis = {'title_text':'Valor Predicho', 'tickmode':'linear'},
  )
  fig_cm.show()


  # curva roc (definido para clasificación binaria)
  fig_roc = None
  if y_score is not None:
    fpr, tpr, thresholds = roc_curve(y_true, y_score, pos_label=pos_label)
    fig_roc = px.area(
        x=fpr, y=tpr,
        title={'text' : f'Curva ROC (AUC={auc(fpr, tpr):.4f})', 'x':0.5, 'xanchor': 'center'},
        labels=dict(x='Ratio Falsos Positivos', y='Ratio Verdaderos Positivos'),
        width=400, height=400
    )
    fig_roc.add_shape(type='line', line=dict(dash='dash'), x0=0, x1=1, y0=0, y1=1)

    fig_roc.update_yaxes(scaleanchor="x", scaleratio=1)
    fig_roc.update_xaxes(constrain='domain')
    
    fig_roc.show()

print('Done!')

Done!


In [23]:
val_blabels

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [109]:
predictions = model.predict(x=val_inputs)

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


In [110]:
pred = np.argmax(predictions["logits"], axis=1)
true = np.argmax(val_blabels, axis=1)

In [111]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [112]:
print(classification_report(true, pred))

              precision    recall  f1-score   support

           0       0.84      0.90      0.87      2952
           1       0.82      0.72      0.77      1848

    accuracy                           0.83      4800
   macro avg       0.83      0.81      0.82      4800
weighted avg       0.83      0.83      0.83      4800



In [113]:
from sklearn.utils.multiclass import unique_labels
import plotly.figure_factory as ff

In [114]:
display_labels = sorted(unique_labels(true, pred), reverse=True)
cm = confusion_matrix(true, pred, labels=display_labels)
z = cm[::-1]
x = display_labels
y =  x[::-1].copy()
z_text = [[str(y) for y in x] for x in z]
fig_cm = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')
fig_cm.update_layout(
    height=400, width=400,
    showlegend=True,
    margin={'t':150, 'l':0},
    title={'text' : 'Matriz de Confusión', 'x':0.5, 'xanchor': 'center'},
    xaxis = {'title_text':'Valor Real', 'tickangle':45, 'side':'top'},
    yaxis = {'title_text':'Valor Predicho', 'tickmode':'linear'},
)
fig_cm.show()
# curva roc (definido para clasificación binaria)


# Nada interesnate por aquí
------



In [32]:
from transformers import TFDistilBertModel

In [87]:
config_bert = DistilBertConfig(num_labels = cfg["num_labels"], 
                               attention_dropout=0.75, seq_classif_dropout=0, dropout=0.75,
                               n_heads=4, dim=128, max_position_embeddings=cfg["max_length"],
                               n_layers=2, hidden_dim=256)
DistilBert = TFDistilBertForSequenceClassification.from_pretrained(cfg["transformer_model_name"], config=config_bert)

Some layers from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing TFDistilBertForSequenceClassification: ['mlm___cls', 'bert']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['pre_classifier', 'dropout_95', 'distilbert', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predic

In [88]:
from tensorflow.keras.layers import Dense, Input, BatchNormalization, LeakyReLU, Softmax, ReLU, Concatenate, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.models import Model

In [97]:
input_ids = Input(shape=(256, ),dtype='int32')
attention_mask = Input(shape=(256, ), dtype='int32')
output_bert = DistilBert(input_ids=input_ids, attention_mask=attention_mask)
output_bert = output_bert['logits']

output = Softmax()(output_bert)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
model_softmax = Model([input_ids, attention_mask], output)
model_softmax.compile(optimizer=optimizer, loss=loss, metrics = [f1Score, 'accuracy'])
model_softmax.summary()

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Model: "functional_27"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_45 (InputLayer)           [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_46 (InputLayer)           [(None, 256)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_for_sequence_cla TFSequenceClassifier 4221570     input_45[0][0]                   
                                                                 input_46[0][0]                   
__________________________________________________________________________________________________
softmax_4 (Softmax)             (None, 2)            0           tf_distil_bert_for_se

In [98]:
cfg['checkpoints_dir'] = 'checkpoints'
cfg['model_name'] = 'distilbert-humor-sm'
cfg['trained_model_name'] = os.path.join(cfg['checkpoints_dir'], cfg['model_name'])

In [99]:
epochs_max = 50
epochs_to_save = 5
batch_size = 32

class_weights = class_weight.compute_class_weight('balanced', np.unique(tlabels), tlabels)
class_weights = dict(enumerate(class_weights))

for epoch in tqdm.tqdm(range(0, epochs_max, epochs_to_save)):
    print(f'Training model, epochs {epoch+1} - {epoch+epochs_to_save}')
    
    # entrenar el modelo. Opcionalmente, se puede suministrar datos de validación => validation_data=(val_inputs,val_blabels_t )
    model_softmax.fit(train_inputs, y=train_blabels_t, epochs=epochs_to_save, 
              batch_size=batch_size, validation_data=(val_inputs, val_blabels_t), class_weight=class_weights)

    #model_softmax.save_pretrained(cfg['trained_model_name'] + f'-epochs-{epoch+1:03d}-{epoch+epochs_to_save:03d}')
    #cfg['tokenizer'].save_pretrained(cfg['trained_model_name'] + f'-epochs-{epoch+1:03d}-{epoch+epochs_to_save:03d}')









  0%|          | 0/10 [00:00<?, ?it/s]The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Training model, epochs 1 - 5
Epoch 1/5


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


599/600 [============================>.] - ETA: 0s - loss: 0.6870 - f1Score: 0.6385 - accuracy: 0.6385

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


600/600 [==============================] - 20s 33ms/step - loss: 0.6870 - f1Score: 0.6386 - accuracy: 0.6386 - val_loss: 0.6699 - val_f1Score: 0.6662 - val_accuracy: 0.6662
Epoch 2/5
600/600 [==============================] - 20s 33ms/step - loss: 0.6815 - f1Score: 0.6394 - accuracy: 0.6394 - val_loss: 0.6696 - val_f1Score: 0.6667 - val_accuracy: 0.6667
Epoch 3/5
369/600 [=================>............] - ETA: 7s - loss: 0.6812 - f1Score: 0.6375 - accuracy: 0.6375

KeyboardInterrupt: ignored